# Building a Spam Filter with Naive Bayes

В этом проекте мы собираемся создать спам-фильтр для SMS-сообщений с использованием полиномиального алгоритма Наивного Байеса. Наша цель - написать программу, которая классифицирует новые сообщения с точностью более 80%, поэтому мы ожидаем, что более 80% новых сообщений будут правильно классифицированы как спам или не спам.

Для обучения алгоритму мы будем использовать набор данных из 5572 SMS-сообщений, которые уже классифицированы людьми. Набор данных был составлен Tiago A. Almeida и José María Gómez Hidalgo, и его можно загрузить с [The UCI Machine Learning Repository.](https://archive.ics.uci.edu/ml/datasets/sms+spam+collection)
Процесс сбора данных более подробно описан [на этой странице.](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/#composition)

## Exploring the Dataset

In [45]:
import pandas as pd

sms_spam = pd.read_csv('SMSSpamCollection', sep='\t', header=None, names=['Label', 'SMS'])

print(sms_spam.shape)
sms_spam.head()

(5572, 2)


,Label,SMS
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [46]:
sms_spam['Label'].value_counts(normalize=True)

ham     0.865937
spam    0.134063
Name: Label, dtype: float64

Мы видим, что около 87% сообщений являются не спамом, а остальные 13% - спамом. Этот пример выглядит репрезентативно, поскольку на практике большинство сообщений, которые получают люди, являются не спамом.

## Training and Test Set

Теперь мы собираемся разделить наш набор данных на обучающий и тестовый набор, где обучающий набор составляет 80% данных, а тестовый набор - на оставшиеся 20%.

In [47]:
# Randomize the dataset
data_randomized = sms_spam.sample(frac=1, random_state=1)
data_randomized.head()

,Label,SMS
1078,ham,"Yep, by the pretty sculpture"
4028,ham,"Yes, princess. Are you going to make me moan?"
958,ham,Welp apparently he retired
4642,ham,Havent.
4674,ham,I forgot 2 ask ü all smth.. There's a card on ...


In [48]:
# Calculate index for split
training_test_index = round(len(data_randomized) * 0.8)

# Training/Test split
training_set = data_randomized[:training_test_index].reset_index(drop=True)
test_set = data_randomized[training_test_index:].reset_index(drop=True)

print(training_set.shape)
print(test_set.shape)

(4458, 2)
(1114, 2)


In [49]:
training_set.head(3)

,Label,SMS
0,ham,"Yep, by the pretty sculpture"
1,ham,"Yes, princess. Are you going to make me moan?"
2,ham,Welp apparently he retired


In [50]:
test_set.head(3)

,Label,SMS
0,ham,Later i guess. I needa do mcat study too.
1,ham,But i haf enuff space got like 4 mb...
2,spam,Had your mobile 10 mths? Update to latest Oran...


Теперь мы проанализируем процентное содержание спама и нежелательных сообщений в учебных и тестовых наборах. Мы ожидаем, что проценты будут близки к тому, что мы имеем в полном наборе данных, где около 87% сообщений являются не спамом, а оставшиеся 13% - спамом.

In [51]:
training_set['Label'].value_counts(normalize=True)

ham     0.86541
spam    0.13459
Name: Label, dtype: float64

In [52]:
test_set['Label'].value_counts(normalize=True)

ham     0.868043
spam    0.131957
Name: Label, dtype: float64

## Data Cleaning

Чтобы вычислить все вероятности, требуемые алгоритмом, нам сначала нужно выполнить небольшую очистку данных, чтобы привести данные в формат, который позволит нам легко извлекать всю необходимую нам информацию.

По сути, мы хотим привести данные в этот формат:

![image](https://dq-content.s3.amazonaws.com/433/cpgp_dataset_3.png)

### Letter Case and Punctuation

Мы начнем с удаления всех знаков препинания и перевода каждой буквы в нижний регистр.

In [53]:
training_set.head()

,Label,SMS
0,ham,"Yep, by the pretty sculpture"
1,ham,"Yes, princess. Are you going to make me moan?"
2,ham,Welp apparently he retired
3,ham,Havent.
4,ham,I forgot 2 ask ü all smth.. There's a card on ...


In [54]:
training_set['SMS'] = training_set['SMS'].str.replace('\W', ' ')
training_set['SMS'] = training_set['SMS'].str.lower()
training_set.head()

,Label,SMS
0,ham,yep by the pretty sculpture
1,ham,yes princess are you going to make me moan
2,ham,welp apparently he retired
3,ham,havent
4,ham,i forgot 2 ask ü all smth there s a card on ...


### Creating the Vocabulary

Давайте теперь перейдем к созданию словаря, который в этом контексте означает список со всеми уникальными словами в нашем обучающем наборе.

In [55]:
training_set['SMS'] = training_set['SMS'].str.split()

vocabulary = []
for sms in training_set['SMS']:
    for word in sms:
        vocabulary.append(word)
        
vocabulary = list(set(vocabulary))

In [56]:
len(vocabulary)

7783

Во всех сообщениях нашего трейниг сета есть 7 783 уникальных слова.

### The Final Training Set

Теперь мы будем использовать словарь, который мы только что создали, для преобразования данных, которое мы хотим.

In [57]:
word_counts_per_sms = {unique_word: [0] * len(training_set['SMS']) for unique_word in vocabulary}

for index, sms in enumerate(training_set['SMS']):
    for word in sms:
        word_counts_per_sms[word][index] += 1

In [58]:
word_counts = pd.DataFrame(word_counts_per_sms)
word_counts.head()

,0,00,000,000pes,008704050406,0089,01223585334,02,0207,02072069400,...,zindgi,zoe,zogtorius,zouk,zyada,é,ú1,ü,〨ud,鈥
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,0,0


In [59]:
training_set_clean = pd.concat([training_set, word_counts], axis=1)
training_set_clean.head()

,Label,SMS,0,00,000,000pes,008704050406,0089,01223585334,02,...,zindgi,zoe,zogtorius,zouk,zyada,é,ú1,ü,〨ud,鈥
0,ham,"[yep, by, the, pretty, sculpture]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ham,"[yes, princess, are, you, going, to, make, me,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ham,"[welp, apparently, he, retired]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ham,[havent],0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ham,"[i, forgot, 2, ask, ü, all, smth, there, s, a,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,0,0


## Calculating Constants First

Теперь мы закончили с очисткой учебного набора и можем приступить к созданию фильтра спама. Наивный алгоритм Байеса должен будет ответить на эти два вероятностных вопроса, чтобы иметь возможность классифицировать новые сообщения:

![formula](https://render.githubusercontent.com/render/math?math=P%28Spam%20%7C%20w_1%2Cw_2%2C%20...%2C%20w_n%29%20%5Cpropto%20P%28Spam%29%20%5Ccdot%20%5Cprod_%7Bi%3D1%7D%5E%7Bn%7DP%28w_i%7CSpam%29&mode=display)

![](https://render.githubusercontent.com/render/math?math=P%28Ham%20%7C%20w_1%2Cw_2%2C%20...%2C%20w_n%29%20%5Cpropto%20P%28Ham%29%20%5Ccdot%20%5Cprod_%7Bi%3D1%7D%5E%7Bn%7DP%28w_i%7CHam%29&mode=display)

Кроме того, чтобы вычислить P (wi | Spam) и P (wi | Ham) внутри приведенных выше формул, нам нужно использовать эти уравнения:

![](https://render.githubusercontent.com/render/math?math=P%28w_i%7CSpam%29%20%3D%20%5Cfrac%7BN_%7Bw_i%7CSpam%7D%20%2B%20%5Calpha%7D%7BN_%7BSpam%7D%20%2B%20%5Calpha%20%5Ccdot%20N_%7BVocabulary%7D%7D&mode=display)

![](https://render.githubusercontent.com/render/math?math=P%28w_i%7CHam%29%20%3D%20%5Cfrac%7BN_%7Bw_i%7CHam%7D%20%2B%20%5Calpha%7D%7BN_%7BHam%7D%20%2B%20%5Calpha%20%5Ccdot%20N_%7BVocabulary%7D%7D&mode=display)

Ниже мы будем использовать наш тренировочный набор для расчета:
- P(Spam) and P(Ham)
- NSpam, NHam, NVocabulary

Мы также будем использовать сглаживание Лапласа и установим![](https://render.githubusercontent.com/render/math?math=%5Calpha%20%3D%201&mode=inline)

In [71]:
# Isolating spam and ham messages first
spam_messages = training_set_clean[training_set_clean['Label'] == 'spam']
ham_messages = training_set_clean[training_set_clean['Label'] == 'ham']

p_spam = len(spam_messages) / len(training_set_clean)
p_ham = len(ham_messages) / len(training_set_clean)

In [78]:
print(round(p_spam, 2))
print(round(p_ham, 2))

0.13
0.87


In [85]:
# N_Spam
n_words_per_spam_message = spam_messages['SMS'].apply(len)
n_spam = n_words_per_spam_message.sum()

# N_Ham
n_words_per_ham_message = ham_messages['SMS'].apply(len)
n_ham = n_words_per_ham_message.sum()

In [86]:
# N_Vocabulary
n_vocabulary = len(vocabulary)

# Laplace smoothing
alpha = 1

## Calculating Parameters

Теперь, когда у нас есть постоянные члены, вычисленные выше, мы можем перейти к вычислению параметров ![](https://render.githubusercontent.com/render/math?math=P%28w_i%7CSpam%29&mode=inline) и ![](https://render.githubusercontent.com/render/math?math=P%28w_i%7CHam%29&mode=inline). Таким образом, каждый параметр будет условным значением вероятности, связанным с каждым словом в словаре.

Параметры рассчитываются по формулам: ![](https://render.githubusercontent.com/render/math?math=P%28w_i%7CSpam%29%20%3D%20%5Cfrac%7BN_%7Bw_i%7CSpam%7D%20%2B%20%5Calpha%7D%7BN_%7BSpam%7D%20%2B%20%5Calpha%20%5Ccdot%20N_%7BVocabulary%7D%7D&mode=display) 
![](https://render.githubusercontent.com/render/math?math=P%28w_i%7CHam%29%20%3D%20%5Cfrac%7BN_%7Bw_i%7CHam%7D%20%2B%20%5Calpha%7D%7BN_%7BHam%7D%20%2B%20%5Calpha%20%5Ccdot%20N_%7BVocabulary%7D%7D&mode=display)

In [87]:
# Initiate parameters
parameters_spam = {unique_word:0 for unique_word in vocabulary}
parameters_ham = {unique_word:0 for unique_word in vocabulary}

# Calculate parameters
for word in vocabulary:
    n_word_given_spam = spam_messages[word].sum()   # spam_messages already defined in a cell above
    p_word_given_spam = (n_word_given_spam + alpha) / (n_spam + alpha*n_vocabulary)
    parameters_spam[word] = p_word_given_spam
    
    n_word_given_ham = ham_messages[word].sum()   # ham_messages already defined in a cell above
    p_word_given_ham = (n_word_given_ham + alpha) / (n_ham + alpha*n_vocabulary)
    parameters_ham[word] = p_word_given_ham

## Classifying A New Message

Теперь, когда мы вычислили все наши параметры, мы можем начать создавать спам-фильтр. Фильтр спама можно понимать как функцию, которая: 
- Принимает в качестве ввода новое сообщение (w1, w2, ..., wn)
- Вычисляет P (Spam | w1, w2, ..., wn) и P (Ham | w1, w2, ..., wn).
- Сравнивает значения P (Спам | w1, w2, ..., wn) и P (Ham | w1, w2, ..., wn) 

In [88]:
import re

def classify(message):
    '''
    message: a string
    '''
    
    message = re.sub('\W', ' ', message)
    message = message.lower().split()
    
    p_spam_given_message = p_spam
    p_ham_given_message = p_ham

    for word in message:
        if word in parameters_spam:
            p_spam_given_message *= parameters_spam[word]
            
        if word in parameters_ham:
            p_ham_given_message *= parameters_ham[word]
            
    print('P(Spam|message):', p_spam_given_message)
    print('P(Ham|message):', p_ham_given_message)
    
    if p_ham_given_message > p_spam_given_message:
        print('Label: Ham')
    elif p_ham_given_message < p_spam_given_message:
        print('Label: Spam')
    else:
        print('Equal proabilities, have a human classify this!')

In [89]:
classify('WINNER!! This is the secret code to unlock the money: C3421.')

P(Spam|message): 1.3481290211300841e-25
P(Ham|message): 1.9368049028589875e-27
Label: Spam


In [90]:
classify("Sounds good, Tom, then see u there")

P(Spam|message): 2.4372375665888117e-25
P(Ham|message): 3.687530435009238e-21
Label: Ham


## Measuring the Spam Filter's Accuracy

Два приведенных выше результата выглядят многообещающими, но давайте посмотрим, насколько хорошо работает фильтр в нашем test dataset, который содержит 1114 сообщений.

Мы начнем с написания функции, которая возвращает классификационные метки вместо их печати.

In [92]:
def classify_test_set(message):    
    '''
    message: a string
    '''
    
    message = re.sub('\W', ' ', message)
    message = message.lower().split()
    
    p_spam_given_message = p_spam
    p_ham_given_message = p_ham

    for word in message:
        if word in parameters_spam:
            p_spam_given_message *= parameters_spam[word]
            
        if word in parameters_ham:
            p_ham_given_message *= parameters_ham[word]
    
    if p_ham_given_message > p_spam_given_message:
        return 'ham'
    elif p_spam_given_message > p_ham_given_message:
        return 'spam'
    else:
        return 'needs human classification'

Теперь, когда у нас есть функция, которая возвращает ярлыки вместо их печати, мы можем использовать ее для создания нового столбца в нашем тестовом наборе.

In [94]:
test_set['predicted'] = test_set['SMS'].apply(classify_test_set)
test_set.head()

,Label,SMS,predicted
0,ham,Later i guess. I needa do mcat study too.,ham
1,ham,But i haf enuff space got like 4 mb...,ham
2,spam,Had your mobile 10 mths? Update to latest Oran...,spam
3,ham,All sounds good. Fingers . Makes it difficult ...,ham
4,ham,"All done, all handed in. Don't know if mega sh...",ham


Теперь мы напишем функцию для измерения точности нашего спам-фильтра, чтобы выяснить, насколько хорошо работает наш спам-фильтр.

In [95]:
correct = 0
total = test_set.shape[0]
    
for row in test_set.iterrows():
    row = row[1]
    if row['Label'] == row['predicted']:
        correct += 1
        
print('Correct:', correct)
print('Incorrect:', total - correct)
print('Accuracy:', correct/total)

Correct: 1100
Incorrect: 14
Accuracy: 0.9874326750448833


Точность близка к 98,74%, что действительно хорошо. Наш спам-фильтр просмотрел 1114 сообщений, которые он не видел во время обучения, и правильно классифицировал 1100 сообщений.

## Conclusion

В этом проекте нам удалось создать спам-фильтр для SMS-сообщений с использованием алгоритма Наивного Байеса. Точность фильтра в тестовом наборе, который мы использовали, составила 98,74%, что является довольно хорошим результатом. Нашей первоначальной целью была точность более 80%, и нам удалось сделать это намного лучше.